In [3]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import requests
import urllib

# ESP32-CAM IP address
esp32cam_url = 'http://192.168.166.99/640x480.jpg'


# Function to fetch images from ESP32-CAM
def get_esp32cam_image():
    try:
        response = requests.get(esp32cam_url, timeout=10)
        if response.status_code == 200:
            img_array = np.array(bytearray(response.content), dtype=np.uint8)
            img = cv2.imdecode(img_array, -1)
            return img
    except Exception as e:
        print(f"Error fetching image from ESP32-CAM: {str(e)}")
    return None


path = 'ImagesBasic'
images = []
classNames = []
mylist = os.listdir(path)
print(mylist)
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)


def find_encodings(images):
    encodeList = []
    i = 0
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
        print(f'Encoding {i}/{len(mylist)} done!')
        i=i+1
    return encodeList


def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')


encodelistknown = find_encodings(images)
print('Encoding Complete!')

['mayank.jpg', 'onkar.jpg', 'prabhat.jpg', 'pranav.jpg', 'sachin.jpg']
['mayank', 'onkar', 'prabhat', 'pranav', 'sachin']
Encoding 0/5 done!
Encoding 1/5 done!
Encoding 2/5 done!
Encoding 3/5 done!
Encoding 4/5 done!
Encoding Complete!


In [2]:
while True:
    # Capture an image from ESP32-CAM
    img = get_esp32cam_image()

    if img is not None:
        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        faceCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

        for encodeFace, faceLoc in zip(encodesCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodelistknown, encodeFace)
            faceDis = face_recognition.face_distance(encodelistknown, encodeFace)
            print(faceDis)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                print(name)
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                markAttendance(name)

        cv2.imshow('ESP32-CAM', img)
        cv2.waitKey(1)

[0.72499978 0.5451248  0.56019464 0.69871153 0.61614889]
ONKAR
[0.7039052  0.52305085 0.5400518  0.668978   0.63411648]
ONKAR
[0.70811464 0.49661038 0.56050385 0.69368158 0.62706918]
ONKAR
[0.7067121  0.5020161  0.53263693 0.70818965 0.63104743]
ONKAR
[0.69831298 0.51528846 0.54029989 0.6916579  0.6122483 ]
ONKAR
[0.6916283  0.52430441 0.520817   0.71103492 0.62583723]
PRABHAT
[0.68826782 0.48497587 0.54108907 0.67792197 0.63080163]
ONKAR
[0.69641263 0.51814803 0.51882722 0.6817595  0.59953738]
ONKAR
[0.70980674 0.50301948 0.5373092  0.69774177 0.61225283]
ONKAR
[0.73292406 0.53843558 0.57817279 0.71562876 0.60342027]
ONKAR
[0.70006324 0.51806949 0.54027592 0.6906406  0.60980978]
ONKAR
[0.69027716 0.52429733 0.52663308 0.66444046 0.59888655]
ONKAR
[0.68887819 0.49915218 0.53661664 0.67413422 0.61598684]
ONKAR
[0.73451241 0.53988322 0.55160471 0.6980005  0.61104962]
ONKAR
[0.68789602 0.53059448 0.50784814 0.67838107 0.60792381]
PRABHAT
[0.73168262 0.52693118 0.55887971 0.7042985  0.6096

KeyboardInterrupt: 